<a href="https://colab.research.google.com/github/thedavidneufeld/ResearchCode/blob/main/RQI_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [1]:
import numpy as np

**Helper Functions**

In [2]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates and returns the conditon number of M
# based on sigma_1/sigma_n
def svd_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-1]

# calculates and returns sigma_1/sigma_{n-1}
def svd_desired_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-2]

# calculates a preconditioned matrix C based on
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1) + np.dot(Y, V.T.conj())
    # return C
    return C

**RQI Algorithm**

In [3]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
def RQI(M, mu, ep, it):
    # create a random vector with norm of 1
    # vector needs to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    # track condition number of (mu)I-M through iterations
    conds = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate the condition number of (mu)I-M
        conds = np.append(conds, svd_cond(muI_M))
        # solve the system ((mu)I-M)x=y
        x = np.linalg.solve(muI_M, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (x^T)Mx
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x) <= ep*np.linalg.norm(M):
            return mu, y, i, conds
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds


**RQI Algorithm with Additive Preconditioning**

In [4]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
#
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # track condition number of (mu)I-M, C, and the desired condition number
    conds = np.empty(0)
    conds_C = np.empty(0)
    conds_desired = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate C
        C = f(mu, M, y, v)
        # calculate the condition numbers
        conds = np.append(conds, svd_cond(muI_M))
        conds_C = np.append(conds_C, svd_cond(C))
        conds_desired = np.append(conds_desired, svd_desired_cond(muI_M))
        # solve the system Cx=y
        x = np.linalg.solve(C, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (y^T)My
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x) <= ep*np.linalg.norm(M):
            return mu, y, i, conds, conds_C, conds_desired
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds, conds_C, conds_desired

**Test and Compare Algorithms**

In [5]:
# Setup

# create a random matrix M
M = np.random.rand(10, 10)
# calculate the eigenvalues/eigenvectors of M
vals, vecs = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract
# a small value to make it an approximation
mu = vals[0]-0.25

In [6]:
# Print condition number of M
svd_cond(M)

224.28717804660735

In [7]:
# Test RQI with 1e-4
result = RQI(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (5.001290331517275+0j)
Eigenvector: [0.25366823+0.j 0.28691931+0.j 0.42649327+0.j 0.37114977+0.j
 0.22410026+0.j 0.3142957 +0.j 0.29682352+0.j 0.38891107+0.j
 0.30533791+0.j 0.22823475+0.j]
Iterations: 3
Condition Numbers: [2.29216988e+01 8.35897777e+02 8.53647992e+05]


In [8]:
# Test RQIAPP with 1e-4
result = RQIAPP(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers for (mu)I-M:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (5.001318607586871+0j)
Eigenvector: [0.25369796+0.j 0.28691431+0.j 0.42645801+0.j 0.37116225+0.j
 0.22411069+0.j 0.31425926+0.j 0.29680148+0.j 0.38893806+0.j
 0.3053653 +0.j 0.22823953+0.j]
Iterations: 2
Condition Numbers for (mu)I-M: [ 22.92169877 495.56281353]
Condition Numbers for C: [4.00489304 3.9085082 ]
Desired Condition Numbers: [1.51002671 1.47859177]


In [9]:
# Test RQI with 1e-6
result = RQI(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (5.00129033150031+0j)
Eigenvector: [-0.25366823+0.j -0.28691931+0.j -0.42649327+0.j -0.37114977+0.j
 -0.22410026+0.j -0.3142957 +0.j -0.29682352+0.j -0.38891107+0.j
 -0.30533791+0.j -0.22823475+0.j]
Iterations: 4
Condition Numbers: [2.29216988e+01 6.36325066e+02 1.95727465e+05 2.72730787e+10]


In [10]:
# Test RQIAPP with 1e-6
result = RQIAPP(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (5.001290332377149+0j)
Eigenvector: [0.25366823+0.j 0.28691931+0.j 0.42649327+0.j 0.37114977+0.j
 0.22410026+0.j 0.3142957 +0.j 0.29682352+0.j 0.38891107+0.j
 0.30533791+0.j 0.22823475+0.j]
Iterations: 3
Condition Numbers: [2.29216988e+01 2.67840771e+02 8.48751533e+04]
Condition Numbers for C: [3.88154691 4.10921262 4.09126723]
Desired Condition Numbers: [1.51002671 1.47985625 1.47709922]


In [11]:
# Test RQI with 1e-8
result = RQI(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: None
Eigenvector: None
Iterations: 100
Condition Numbers: [2.29216988e+01 6.67725292e+02 4.05332705e+05 1.31567163e+11
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15 9.13922613e+15 4.48881446e+15 9.13922613e+15
 4.48881446e+15

In [12]:
# Test RQIAPP with 1e-8
result = RQIAPP(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (5.001290336366048+0j)
Eigenvector: [0.25366823+0.j 0.28691931+0.j 0.42649327+0.j 0.37114977+0.j
 0.22410026+0.j 0.31429569+0.j 0.29682352+0.j 0.38891107+0.j
 0.30533791+0.j 0.22823475+0.j]
Iterations: 3
Condition Numbers: [2.29216988e+01 2.21111470e+02 4.14075702e+04]
Condition Numbers for C: [3.92871714 3.8171436  3.81678642]
Desired Condition Numbers: [1.51002671 1.48043868 1.47709014]


In [13]:
# Print actual eigenvalue/eigenvector
print(vals[0], vecs[:,0])

(5.00129033150031+0j) [-0.25366823+0.j -0.28691931+0.j -0.42649327+0.j -0.37114977+0.j
 -0.22410026+0.j -0.3142957 +0.j -0.29682352+0.j -0.38891107+0.j
 -0.30533791+0.j -0.22823475+0.j]
